In [1]:
import pandas as pd
import random
import numpy as np
import matplotlib.pyplot as plt
import datetime
import seaborn as sns
import requests
from scipy.stats import norm, chi2,gumbel_l
from concurrent.futures import ThreadPoolExecutor, as_completed
from multiprocessing import Pool, cpu_count

import ipywidgets as widgets
from ipydatagrid import DataGrid, TextRenderer
from IPython.display import display,Markdown
from IPython.display import HTML
import plotly.express as px
import plotly.graph_objects as go

from Git import *
from Binance_API import BinanceAPI
from RiskMetrics import *
from PnL_Computation import PnL
from Stock_Data import get_close
from Rebalancing import *
from Metrics import *

In [5]:
def display_crypto_app():
    # --- strategy dictionary ---
    dico_strategies = {
        'Minimum Variance': 'minimum_variance',
        'Risk Parity': 'risk_parity',
        'Sharpe Ratio': 'sharpe_ratio',
        'Maximum Diversification':'maximum_diversification'}
    
    
    options_strat = list(dico_strategies.keys())

    # --- globals ---
    global tickers_dataframe, tickers, dataframe, returns_to_use, prices
    global rolling_optimization, performance_pct, performance_fund, dates_end,quantities,cumulative_results,global_returns
    global book_cost,realized_pnl,profit_and_loss,holding_tickers,current_weights,fund_names,grid,trades
    
    tickers_dataframe = Binance.get_market_cap().set_index('Ticker')

    tickers = []
    holding_tickers=[]
    dataframe = pd.DataFrame()
    cumulative_results=pd.DataFrame()
    global_returns=pd.DataFrame()

    fund_names=[]
    current_weights=pd.DataFrame()
    book_cost=pd.DataFrame()
    realized_pnl=pd.DataFrame()
    trades=pd.DataFrame()

    profit_and_loss=pd.DataFrame()
    returns_to_use = pd.DataFrame()
    prices = pd.DataFrame()
    
    rolling_optimization = pd.DataFrame()
    quantities=pd.DataFrame()
    
    performance_pct = pd.DataFrame()
    performance_fund = pd.DataFrame()
    
    dates_end = []
    constraint_container = {'constraints': [], 'allocation_df': pd.DataFrame()}

    # --- UI setup ---
    start_date = widgets.DatePicker(
        value=datetime.date(2020, 1, 1),
        description='Starting Date of Backtest',
        style={'description_width': '200px'},
        layout=widgets.Layout(width='350px')
    )

    n_crypto = widgets.IntSlider(
        min=1, max=100, value=20,
        description='Number of Crypto',
        style={'description_width': '200px'},
        layout=widgets.Layout(width='500px')
    )
    
    loading_bar = widgets.IntProgress(description='Loading prices...',min=0, max=100,style={'description_width': '150px'})
    
    data_button = widgets.Button(description='Get Prices', button_style='info')
    scope_output = widgets.Output()
    strategy_output = widgets.Output()
    main_output = widgets.Output()
    output_returns = widgets.Output()
    constraint_output = widgets.Output()
    
    dropdown_asset1 = widgets.Dropdown(description='Asset 1',style={'description_width': '150px'})
    dropdown_asset2 = widgets.Dropdown(description='Asset 2',style={'description_width': '150px'} )

    
    def scope_update(n):
        nonlocal scope_output
        global tickers_dataframe, tickers

        try:
            selected_tickers=tickers_dataframe.iloc[:n]
            selected_tickers_list = list(set(selected_tickers.index))
            
        except Exception as e:
            with scope_output:
                scope_output.clear_output(wait=True)
                print("Error fetching market caps:", e)
            return
  
              
        with scope_output:
            scope_output.clear_output(wait=True)
            display(display_scrollable_df(selected_tickers))
    
            checkboxes = {
                t: widgets.Checkbox(description=t, value=True)
                for t in selected_tickers_list
            }
    
            rows = [
                widgets.HBox(list(checkboxes.values())[i:i+5])
                for i in range(0, len(checkboxes), 5)
            ]
    
            ui = widgets.VBox(rows)
    
            def on_change(change=None):
                selected = [k for k, cb in checkboxes.items() if cb.value]
                # update global tickers
                global tickers
                tickers = selected
    
                df = pd.DataFrame(selected, columns=["Selected Tickers"]).set_index("Selected Tickers")
    
            for cb in checkboxes.values():
                cb.observe(on_change, names="value")
    
            on_change()

            display(Markdown("### Selected Tickers"))

            display(ui)
            
    scope_update(n_crypto.value)
    
    n_crypto.observe(lambda ch: scope_update(ch['new']) if ch['name'] == 'value' else None, names='value')
    
    price_output=widgets.Output()

    def get_price_threading(tickers,start_date):
            
        today = datetime.date.today()
        days_total = (today - start_date).days
        if days_total <= 0:
            print("Start date must be in the past.")
            return

        remaining = days_total % 500
        numbers_of_table = days_total // 500

        loading_bar.value = 0
        display(loading_bar)
        loading_bar.max = numbers_of_table + 1
        
        start_dt= datetime.datetime.combine(start_date, datetime.time())
        end_dates = [
                start_dt + datetime.timedelta(days=500 * i)
                for i in range(numbers_of_table + 1)
            ]
    
        end_dates.append(
            datetime.datetime.combine(
                today - datetime.timedelta(days=remaining),
                datetime.time()
            )
        )

        def fetch_prices(end_date):
            return Binance.get_price(tickers, end_date)

        price = None

        try:
            with ThreadPoolExecutor(max_workers=8) as executor:
                futures = [executor.submit(fetch_prices, d) for d in end_dates]

                for future in as_completed(futures):
                    data = future.result()

                    if price is None:
                        price = data
                    else:
                        price = price.combine_first(data)

                    loading_bar.value += 1

        except Exception as e:
            print("❌ Error while fetching prices:", e)
            return

        price = price.sort_index()
        price = price[~price.index.duplicated(keep="first")]
        price.index = pd.to_datetime(price.index)
        
        loading_bar.value = loading_bar.max

        return price

        
    def get_prices(_=None):
        
        global prices, dataframe, returns_to_use, dates_end, valid_cols_model
    
        get_holdings(None)
        combined_tickers = sorted(list(set(tickers + holding_tickers)))
    
        with main_output:
            main_output.clear_output(wait=True)
    
            if not tickers:
                print("No tickers available. Please fetch tickers first.")
                return
    
            start = start_date.value
            if not isinstance(start, datetime.date):
                print("Please select a valid start date.")
                return

            
            scope_prices=get_price_threading(combined_tickers,start)
    
            prices = scope_prices.loc[:, scope_prices.columns != "USDCUSDT"]
    
            returns = np.log(1 + prices.pct_change(fill_method=None))
            returns.index = pd.to_datetime(returns.index)
    
            valid_cols = returns.columns[returns.isna().sum() < 30]
            returns_to_use = returns[valid_cols].sort_index()
    
            dataframe = prices[valid_cols].sort_index().dropna()
            dataframe.index = pd.to_datetime(dataframe.index)
            returns_to_use = returns_to_use[~returns_to_use.index.duplicated(keep="first")]
        
            main_output.clear_output()
    
            dropdown_asset.options = list(dataframe.columns) + ["All"]
    
            dropdown_asset1.options = dataframe.columns
            dropdown_asset1.value = dataframe.columns[0]
    
            dropdown_asset2.options = dataframe.columns
            dropdown_asset2.value = dataframe.columns[1]
    
            print(
                f"✅ Loaded prices for {len(dataframe.columns)} assets "
                f"from {dataframe.index[0].date()} to {dataframe.index[-1].date()}"
            )
    
            asset_risk = get_asset_risk(dataframe)
            asset_returns = get_asset_returns(dataframe)
            
            display(display_scrollable_df(asset_returns))
            display(display_scrollable_df(asset_risk))
                        
        with price_output:
            
            price_output.clear_output(wait=True)

            price_output_graph=widgets.Output()
            return_output_graph=widgets.Output()
            
            with price_output_graph:
                fig = px.line(
                    dataframe.loc[start_date_perf.value:end_date_perf.value],
                    title="Price",
                    width=800,
                    height=400
                , render_mode = 'svg')
                fig.update_layout(plot_bgcolor="black", paper_bgcolor="black", font_color="white")
                fig.update_traces(visible="legendonly", selector=lambda t: t.name != "BTCUSDT")
                fig.show()

            with return_output_graph:
                
                cumulative_returns = returns_to_use.loc[start_date_perf.value:end_date_perf.value].copy()
                cumulative_returns.iloc[0] = 0
                cumulative_returns = (1 + cumulative_returns).cumprod() * 100
                fig2 = px.line(
                    cumulative_returns,
                    title="Cumulative Performance",
                    width=800,
                    height=400
                , render_mode = 'svg')
                fig2.update_layout(plot_bgcolor="black", paper_bgcolor="black", font_color="white")
                fig2.update_traces(visible="legendonly", selector=lambda t: t.name != "BTCUSDT")
                fig2.show()

            ui=widgets.HBox([price_output_graph,return_output_graph])
            display(ui)
            display(display_scrollable_df(dataframe))

    data_button.on_click(get_prices)
    start_date.observe(lambda ch: get_prices() if ch['name'] == 'value' and ch['new'] else None, names='value')
    

    # --- constraint UI ---
    dropdown_asset = widgets.Dropdown(description='Asset:', options=['All'], value=None)
    dropdown_sign = widgets.Dropdown(description='Sign:', options=["=", "≥", "≤"])
    dropdown_limit = widgets.FloatText(description='Limit')
    add_constraint_btn = widgets.Button(description='Add Constraint', button_style='success')
    clear_constraints_btn = widgets.Button(description='Clear All', button_style='danger')
    constraints = []
    
    
    selected_fund = widgets.Dropdown(description="Fund:")
    selected_bench = widgets.Dropdown(description="Bench:")
    selected_fund_var = widgets.Dropdown(description="Fund:")
    
    def on_add_constraint_clicked(_):
        constraints.append({
            'Asset': dropdown_asset.value,
            'Sign': dropdown_sign.value,
            'Limit': dropdown_limit.value
        })
        with constraint_output:
            constraint_output.clear_output(wait=True)
            display(display_scrollable_df(pd.DataFrame(constraints)))

    def on_clear_constraints(_):
        constraints.clear()
        with constraint_output:
            constraint_output.clear_output(wait=True)
            display(display_scrollable_df(pd.DataFrame(columns=['Asset', 'Sign', 'Limit'])))

    add_constraint_btn.on_click(on_add_constraint_clicked)
    clear_constraints_btn.on_click(on_clear_constraints)
    
    def on_add_click(b):
        
        global fund_names,grid

        if grid.data is None or grid.data.empty:
            return
        new_row = np.zeros(dataframe.shape[1])
        label = f"Allocation {grid.data.shape[0]}"
        new_df = pd.DataFrame([new_row], columns=grid.data.columns, index=[label])
        updated_df = pd.concat([pd.DataFrame(grid.data), new_df])
        grid.data = updated_df
        
        benchmark_tracking_error.options=grid.data.index
        selected_fund.options=grid.data.index
        selected_bench.options=grid.data.index
        
        selected_fund_var.options=grid.data.index

    def clear_allocation(b):
        
        nonlocal constraint_container
        if constraint_container.get('allocation_df') is not None:
            grid.data = constraint_container['allocation_df']
            
    button_add = widgets.Button(description="Add Allocation")    
    button_clear = widgets.Button(description="Clear Allocation")            
    button_add.on_click(on_add_click)
    button_clear.on_click(clear_allocation)
    
    # --- date pickers for performance ---
    if isinstance(start_date.value, datetime.date):
        sd = start_date.value
        start_perf_date = datetime.date(sd.year, sd.month + 2, 1)
    else:
        start_perf_date = datetime.date.today() - datetime.timedelta(days=365)

    start_date_perf = widgets.DatePicker(value=start_perf_date, layout=widgets.Layout(width='350px'))
    end_date_perf = widgets.DatePicker(value=datetime.date.today(), layout=widgets.Layout(width='350px'))
    
    frequency_graph=widgets.Dropdown(description='Frequency:', options=['Year','Month'], value='Year')
    benchmark=widgets.Dropdown(description='Benchmark:', options=['Fund','Bitcoin'], value='Bitcoin')
    fund=widgets.Dropdown(description='Fund:', options=['Fund','Bitcoin'], value='Fund')
    benchmark_tracking_error=widgets.Dropdown(description='Benchmark:')

    perf_output=widgets.Output()
    vol_output=widgets.Output()
    drawdown_output=widgets.Output()
    frontier_output=widgets.Output()
    
    # --- performance update ---
    def updated_cumulative_perf(_):
        
        global performance_pct, performance_fund,cumulative_results,global_returns
        
        try:
            start_ts = pd.to_datetime(start_date_perf.value)
            end_ts = pd.to_datetime(end_date_perf.value)
        except Exception:
            with output_returns:
                output_returns.clear_output(wait=True)
                print("⚠️ Invalid start/end dates.")
            return
            
        if dataframe.empty:
            with main_output:
                main_output.clear_output(wait=True)
                print("⚠️ Load Prices.")
                return
        else:

            with main_output:
                
                main_output.clear_output(wait=True)
                range_prices=dataframe.loc[start_ts:end_ts]
                range_returns=range_prices.pct_change()
                
                asset_risk=get_asset_risk(range_prices)
                asset_returns=get_asset_returns(range_prices)
                display(display_scrollable_df(asset_returns))
                display(display_scrollable_df(asset_risk))
                
            with price_output:
                price_output.clear_output(wait=True)
                price_output_graph=widgets.Output()
                return_output_graph=widgets.Output()
                
                with price_output_graph:
                    fig = px.line(dataframe.loc[start_date_perf.value:end_date_perf.value], title='Price', width=800, height=400, render_mode = 'svg')
                    fig.update_layout(plot_bgcolor="black", paper_bgcolor="black", font_color="white")
                    fig.update_traces(textfont=dict(family="Arial Narrow", size=15))
                    fig.update_traces(visible="legendonly", selector=lambda t: not t.name in ["BTCUSDT"])
        
                    fig.show()
                with return_output_graph:
                    
                    cumulative_returns=returns_to_use.loc[start_date_perf.value:end_date_perf.value].copy()
                    cumulative_returns.iloc[0]=0
                    cumulative_returns=(1+cumulative_returns).cumprod()*100
                    
                    fig2 = px.line(cumulative_returns, title='Cumulative Performance', width=800, height=400, render_mode = 'svg')
                    fig2.update_layout(plot_bgcolor="black", paper_bgcolor="black", font_color="white")
                    fig2.update_traces(textfont=dict(family="Arial Narrow", size=15))
                    fig2.update_traces(visible="legendonly", selector=lambda t: not t.name in ["BTCUSDT"])
        
                    fig2.show()
    
                ui=widgets.HBox([price_output_graph,return_output_graph])
                display(ui)
                display(display_scrollable_df(dataframe))

        if performance_pct is None or performance_pct.empty:
            with output_returns:
                output_returns.clear_output(wait=True)
                print("⚠️ No performance data available yet. Please run an optimization first.")
            return
       
        if pd.isna(start_ts) or pd.isna(end_ts) or start_ts > end_ts:
            
            with main_output:
                main_output.clear_output()
                print("⚠️ Invalid date range.")
            with output_returns:
                output_returns.clear_output()
                print("⚠️ Invalid date range.")
            with vol_output:
                vol_output.clear_output()
            with perf_output:
                perf_output.clear_output()
            with drawdown_output:
                drawdown_output.clear_output()
            with frontier_output:
                frontier_output.clear_output()
                
            return


        performance_pct.index = pd.to_datetime(performance_pct.index)

        cumulative_performance = performance_pct.loc[start_ts:end_ts]

        if cumulative_performance.empty:
            available_start = performance_pct.index.min().date()
            available_end = performance_pct.index.max().date()
            with output_returns:
                output_returns.clear_output(wait=True)
                print(f"⚠️ No data found for this date range. Available range: {available_start} → {available_end}")
            return

        # cumulative_performance = cumulative_performance.copy()

            
        cumulative_performance.iloc[0] = 0
        cumulative_results = (1 + cumulative_performance).cumprod() * 100
        
        portfolio_returns = rebalanced_time_series(range_prices, grid.data, frequency=rebalancing_frequency.value)
        cumulative_results=pd.concat([cumulative_results,portfolio_returns],axis=1)
        global_returns=cumulative_results.pct_change()
                
        drawdown = (cumulative_results - cumulative_results.cummax()) / cumulative_results.cummax()
        rolling_vol_ptf=cumulative_results.pct_change().rolling(window_vol.value).std()*np.sqrt(260)
        frontier_indicators, fig4 = get_frontier(range_returns, grid.data)
        update_dropdown_options()
        

        with output_returns:
            output_returns.clear_output(wait=True)
            display(display_scrollable_df(rebalanced_metrics(cumulative_results)))
            display(display_scrollable_df(get_portfolio_risk(grid.data, range_prices, cumulative_results, benchmark_tracking_error.value)))
            display(display_scrollable_df(frontier_indicators))

        with perf_output:
            
            perf_output.clear_output(wait=True)
            fig = px.line(cumulative_results, title='Performance', width=800, height=400, render_mode = 'svg')
            fig.update_layout(plot_bgcolor="black", paper_bgcolor="black", font_color="white")
            fig.update_traces(visible="legendonly", selector=lambda t: not t.name in ["Fund","Bitcoin"])
            fig.update_traces(textfont=dict(family="Arial Narrow", size=15))

            fig.show()
        with drawdown_output:
            
            drawdown_output.clear_output(wait=True)

            fig2 = px.line(drawdown, title='Drawdown', width=800, height=400, render_mode = 'svg')
            fig2.update_layout(plot_bgcolor="black", paper_bgcolor="black", font_color="white")
            fig2.update_traces(visible="legendonly", selector=lambda t: not t.name in ["Fund","Bitcoin"])
            fig2.update_traces(textfont=dict(family="Arial Narrow", size=15))

            fig2.show()

        with vol_output:
            vol_output.clear_output(wait=True)

            fig3 = px.line(rolling_vol_ptf, title="Portfolio Rolling Volatility", render_mode = 'svg').update_traces(visible="legendonly", selector=lambda t: not t.name in ["Fund","Bitcoin"])
            fig3.update_layout(plot_bgcolor="black", paper_bgcolor="black", font_color="white", width=800, height=400) 
            fig3.update_traces(visible="legendonly", selector=lambda t: not t.name in ["Fund","Bitcoin"])
            fig3.update_traces(textfont=dict(family="Arial Narrow", size=15))

            fig3.show()
        with frontier_output:
            frontier_output.clear_output(wait=True)

            fig4.update_layout(width=800, height=400,title={'text': "Efficient Frontier"})
            fig4.update_traces(textfont=dict(family="Arial Narrow", size=15))

            fig4.show()
            # display(display_scrollable_df(cumulative_results))
            # display(display_scrollable_df(drawdown))

        
    # start_date_perf.observe(updated_cumulative_perf)
    # end_date_perf.observe(updated_cumulative_perf)
    refresh_perf_button=widgets.Button(description='Refresh')
    refresh_perf_button.on_click(updated_cumulative_perf)
    
    # --- optimization ---
    rebalancing_frequency = widgets.Dropdown(description='Frequency', options=['Monthly', 'Quarterly', 'Yearly'], value='Monthly')
    strat = widgets.Dropdown(description='Strategy', options=options_strat, value='Minimum Variance')
    window_vol=widgets.IntText(
    value=252,
    description='Vol Window:',
    disabled=False
    )    
    calendar_output=widgets.Output()
    
    def show_graph(_):
        with calendar_output:
            
            calendar_output.clear_output(wait=True)
            
            if cumulative_results.empty or cumulative_results.shape[1]<2:
                print("⚠️ No performance data available yet. Please run an optimization first.")
                return

            if fund.value==benchmark.value:
                print("⚠️ Benchmark and Fund must be different.")
                return
            
            graphs=get_calendar_graph(cumulative_results, 
                               freq=frequency_graph.value, 
                               benchmark=benchmark.value, 
                               fund=fund.value)
            
            return_and_vol_graph=widgets.Output()
            sharpe_and_te_graph=widgets.Output()
            keys=list(graphs.keys())
            
            with return_and_vol_graph:
                graphs[keys[0]].show()
                graphs[keys[2]].show()
            with sharpe_and_te_graph:
                graphs[keys[1]].show()
                graphs[keys[3]].show()   
            
            ui=widgets.HBox([return_and_vol_graph,sharpe_and_te_graph])
            display(ui)
            
    graph_button=widgets.Button(description='Update Perf',button_style='info')
    graph_button.on_click(show_graph)

    optimize_btn = widgets.Button(description='Optimize Portfolio', button_style='primary')
    grid = DataGrid(pd.DataFrame(), editable=True, layout={"height": "250px"})
    

    def update_dropdown_options():
        """Safely refresh fund/benchmark dropdowns after cumulative_results is updated."""
        if 'cumulative_results' not in globals() or cumulative_results.empty:
            return

        global fund_names
        
        options = list(cumulative_results.columns)
    
        # Fund options exclude the currently selected benchmark
    
        fund.options = options
        benchmark.options = options
        
        fund.value = options[0]
        benchmark.value = options[1]
        fund_names=list(grid.data.index)
        benchmark_tracking_error.options=grid.data.index

        selected_fund.options=grid.data.index

        selected_bench.options=grid.data.index

        
        selected_fund_var.options=grid.data.index
        
    def on_optimize_clicked(_):
        global fund_names,grid

        with constraint_output:
            constraint_output.clear_output(wait=True)
            if dataframe.empty or returns_to_use.empty:
                print("⚠️ Load price data before optimizing.")
                return
            constraint_df = pd.DataFrame(constraints)
            cons = None
            if not constraint_df.empty:
                try:
                    cons = build_constraint(dataframe, constraint_df.to_numpy())
                except Exception as e:
                    print("Error building constraints:", e)
                    
            portfolio = RiskAnalysis(returns_to_use.loc[start_date_perf.value:end_date_perf.value])
            sharpe = portfolio.optimize("sharpe_ratio")
            minvar = portfolio.optimize("minimum_variance")
            rp = portfolio.optimize("risk_parity")
            max_div=portfolio.optimize("maximum_diversification")
            
            sharpe_c = minvar_c = rp_c =max_div_c= None
            equal_weights = np.ones(returns_to_use.shape[1]) / returns_to_use.shape[1]
            
            if cons is not None:
                sharpe_c = portfolio.optimize("sharpe_ratio", constraints=cons)
                minvar_c = portfolio.optimize("minimum_variance", constraints=cons)
                rp_c = portfolio.optimize("risk_parity", constraints=cons)
                max_div_c=portfolio.optimize("maximum_diversification",constraints=cons)
                
            allocation = {
                'Optimal Portfolio': sharpe.tolist(),
                'Constrained Optimal Portfolio': sharpe_c.tolist() if sharpe_c is not None else sharpe.tolist(),
                'Min Variance': minvar.tolist(),
                'Constrained Min Var': minvar_c.tolist() if minvar_c is not None else minvar.tolist(),
                'Max Diversification':max_div.tolist(),
                'Max Diversification Constrained':max_div_c.tolist() if max_div_c is not None else max_div.tolist(),
                'Risk Parity': rp.tolist(),
                'Constrained RP': rp_c.tolist() if rp_c is not None else rp.tolist(),
                'Equal Weighted':equal_weights.tolist()}

            allocation_df = pd.DataFrame(allocation, index=dataframe.columns).T.round(4)
            if set(current_weights.index).issubset(dataframe.columns):
                allocation_df = allocation_df.combine_first(current_weights.T).fillna(0)
            
            constraint_container = {'constraints': constraints, 'allocation_df': allocation_df}
            grid.data = allocation_df
            
            benchmark_tracking_error.options=grid.data.index
            benchmark_tracking_error.value=grid.data.index[0]

            selected_fund.options=grid.data.index
            selected_fund.value=grid.data.index[0]
    
            selected_bench.options=grid.data.index
            selected_bench.value=grid.data.index[0]
    
            
            selected_fund_var.options=grid.data.index
            selected_fund_var.value=grid.data.index[0]
            
            with constraint_output:
                constraint_output.clear_output(wait=True)
                display(display_scrollable_df(pd.DataFrame(constraints))) 

    def get_result(_):
        nonlocal constraint_container
        global rolling_optimization, performance_pct, performance_fund, dates_end, quantities
    
        with strategy_output:
            strategy_output.clear_output(wait=True)
            if dataframe.empty or returns_to_use.empty:
                print("⚠️ Load price data before optimizing.")
                return
    
            # Build constraints
            cons = None
            if constraints:
                try:
                    cons = build_constraint(dataframe, pd.DataFrame(constraints).to_numpy())
                except Exception as e:
                    print("Error building constraints:", e)
    
            # Candidate anchors
            freq_map = {
                'Monthly': pd.offsets.BMonthEnd(),
                'Quarterly': pd.offsets.BQuarterEnd(),
                'Yearly': pd.offsets.BYearEnd()
            }
            offset = freq_map.get(rebalancing_frequency.value, pd.offsets.BMonthEnd())
            candidate_anchors = pd.DatetimeIndex(sorted(set(dataframe.index + offset)))
            if candidate_anchors.empty:
                candidate_anchors = pd.DatetimeIndex([returns_to_use.index[-1]])
    
            idx = returns_to_use.index.get_indexer(candidate_anchors, method='nearest')
            idx = np.array(idx)
            idx = idx[idx >= 0]
            selected_dates = returns_to_use.index[idx].tolist()
            dates_end = sorted(list(set(selected_dates + [returns_to_use.index[-1]])))
    
            if len(dates_end) < 2:
                print("⚠️ Not enough anchor dates to perform rolling optimization.")
                return
    
            # Prepare tasks
            strategy_key = dico_strategies[strat.value]
            tasks = [(returns_to_use.loc[dates_end[i]:dates_end[i+1]],dates_end[i], dates_end[i+1]) for i in range(len(dates_end)-1)]
    
            # Run with threads
            results = {}
            def worker(subset,start, end):

                if subset.empty or len(subset) < 2:
                    return None
                try:
                    risk = RiskAnalysis(subset)
                    if cons:
                        opt = risk.optimize(objective=strategy_key, constraints=cons)
                    else:
                        opt = risk.optimize(objective=strategy_key)
                    return subset.index[-1], np.round(opt, 6)
                except Exception:
                    return None
    
            with ThreadPoolExecutor(max_workers=cpu_count()) as executor:
                futures = {executor.submit(worker,subset, start, end): (subset,start, end) for subset,start, end in tasks}
                for future in as_completed(futures):
                    out = future.result()
                    if out is not None:
                        date_key, weights = out
                        results[date_key] = weights
            if not results:
                print("⚠️ No valid optimizations computed.")
                return
    
            rolling_optimization = pd.DataFrame(results, index=dataframe.columns).T.sort_index()
            if not rolling_optimization.empty:
                first_row = pd.Series(1 / len(dataframe.columns), index=dataframe.columns, name=dates_end[0])
                rolling_optimization = pd.concat([pd.DataFrame([first_row]), rolling_optimization])
    
            display(display_scrollable_df(rolling_optimization))
            
            model=pd.DataFrame(rolling_optimization.iloc[-2])
            model.columns=['Model']
            if not 'Model' in grid.data.index:
                grid.data=pd.concat([grid.data,model.T],axis=0)
                
            quantities = rebalanced_dynamic_quantities(dataframe, rolling_optimization)
            performance_fund = pd.DataFrame({'Fund': (quantities * dataframe).sum(axis=1)})
            if 'BTCUSDT' in dataframe.columns:
                performance_fund['Bitcoin'] = dataframe['BTCUSDT']
            performance_pct = performance_fund.pct_change(fill_method=None)
            
            cumulative=(1+performance_pct).cumprod()*100
            drawdown=pd.DataFrame((cumulative-cumulative.cummax()))/cumulative.cummax()            
            date_drawdown=drawdown.idxmin().dt.date
            max_drawdown=drawdown.min()
            
            metrics={}
            metrics['Tracking Error']=(performance_pct['Fund']-performance_pct['Bitcoin']).std()*np.sqrt(252)
            metrics['Fund Vol']=performance_pct['Fund'].std()*np.sqrt(252)
            metrics['Bitcoin Vol']=performance_pct['Bitcoin'].std()*np.sqrt(252)
            metrics['Fund Return']=performance_fund['Fund'].iloc[-2]/performance_fund['Fund'].iloc[0]
            metrics['Bitcoin Return']=performance_fund['Bitcoin'].iloc[-2]/performance_fund['Bitcoin'].iloc[0]
            metrics['Sharpe Ratio']=(1+metrics['Fund Return'])**(1/len(set(returns_to_use.index.year)))/metrics['Fund Vol']
            metrics['Bitcoin Sharpe Ratio']=(1+metrics['Bitcoin Return'])**(1/len(set(returns_to_use.index.year)))/metrics['Bitcoin Vol']

            metrics['Fund Drawdown']=max_drawdown['Fund']
            metrics['Bitcoin Drawdown']=max_drawdown['Bitcoin']
            
            metrics['Fund Date Drawdown']=date_drawdown['Fund']
            metrics['Bitcoin Date Drawdown']=date_drawdown['Bitcoin']
            
            indicators=pd.DataFrame(metrics.values(),index=metrics.keys(),columns=['Indicators'])
            # show results
            display(display_scrollable_df(indicators.round(4)))
            updated_cumulative_perf(None)
            show_graph(None)
            get_holdings(None)
            
    optimize_btn.on_click(on_optimize_clicked)
    results_button=widgets.Button(description='Get Results',button_style='info')
    results_button.on_click(get_result)

    positions_output=widgets.Output()
    holding_output=widgets.Output()
    loading_bar_pnl = widgets.IntProgress(description='Loading P&L...',min=0, max=100,style={'description_width': '150px'})
    def get_holdings(_):

        global holding_tickers,current_weights,pnl
        
        quantities_api=Binance.binance_api.user_asset()
        current_quantities=pd.DataFrame(quantities_api).sort_values(by='free',ascending=False)
        current_quantities['asset']=current_quantities['asset']+'USDT'
        current_quantities=current_quantities.set_index('asset')
        
        current_positions=Binance.get_inventory().round(4)
        current_positions.columns=['Current Portfolio in USDT','Current Weights']
        amount=current_positions.loc['Total']['Current Portfolio in USDT']
        condition=current_positions.index!='Total'

        holding_tickers=current_positions.index[condition]
        holding_tickers=holding_tickers.to_list()
        
        inventory_weights=(current_positions['Current Weights'].apply(lambda x: np.round(x,4))).to_dict()
        inventory_weights.pop('Total')
        inventory_weights.pop('USDCUSDT')
        
        if "USDTUSDT" in holding_tickers:
            inventory_weights.pop('USDTUSDT')
        else: 
            pass
            
        current_weights=pd.DataFrame(inventory_weights.values(),index=inventory_weights.keys(),columns=['Current Weights'])
                
        with positions_output:
            positions_output.clear_output(wait=True)
        
            if dataframe.empty or returns_to_use.empty:
                print("⚠️ Load Prices.")
        
            elif quantities.empty:
                print("⚠️ Load Model.")
        
            else:
                last_prices = Binance.get_price(list(quantities.iloc[-1].keys()))
                positions = pd.DataFrame(quantities.iloc[-1] * last_prices).T
        
                amount_ex_out_of_positions = (
                    current_positions.loc[
                        ~(current_positions.index.isin(positions.index) | (current_positions.index == 'Total')),
                        'Current Portfolio in USDT'
                    ].sum()
                )
        
                positions['Weights Model'] = positions / positions.sum()
                positions['Model (without out of Model Positions)'] = (
                    positions['Weights Model'] * (amount - amount_ex_out_of_positions)
                )
                positions['Model'] = positions['Weights Model'] * amount
        
                portfolio = pd.concat(
                    [positions[['Model', 'Model (without out of Model Positions)', 'Weights Model']],
                     current_positions.loc[condition]],
                    axis=1
                ).fillna(0)
        
                portfolio['Spread'] = portfolio['Current Portfolio in USDT'] - portfolio['Model']
                portfolio.loc['Total'] = portfolio.sum(axis=0)
                portfolio = (
                    portfolio.loc[~(portfolio == 0).all(axis=1)]
                    .sort_values(by='Weights Model', ascending=False)
                    .round(4)
                )
        
                display(display_scrollable_df(portfolio))
        
        with holding_output:
            holding_output.clear_output(wait=True)
        
            if book_cost.empty and realized_pnl.empty:
                display(display_scrollable_df(current_positions))
                print("⚠️ P&L not Computed.")
            else:
                last_book_cost = book_cost.iloc[-1] if not book_cost.empty else pd.Series(dtype=float)
                realized_pnl_filled = realized_pnl if not realized_pnl.empty else pd.Series(dtype=float)
        
        
                pnl = pd.concat(
                    [last_book_cost, last_book_cost, current_positions.loc[condition], realized_pnl_filled],
                    axis=1
                )
                pnl.columns = ['Average Cost', 'Book Cost', 'Price in USDT', 'Weights', 'Realized P&L']
        
                pnl['Book Cost'] = (pnl['Book Cost'] * current_quantities['free'].astype(float)).fillna(0)
                pnl['Unrealized P&L'] = (pnl['Price in USDT'] - pnl['Book Cost']).round(2)
                pnl = pnl.fillna(0)
                pnl['Weights'] = pnl['Weights'].round(4)
        
                pnl['Total P&L'] = pnl['Unrealized P&L'] #+ pnl['Realized P&L']
                pnl.loc['Total'] = pnl.sum()
                pnl.loc['Total', 'Average Cost'] = np.nan
                pnl.loc['Total', 'Book Cost'] = pnl.loc['Total', 'Price in USDT'] - pnl.loc['Total', 'Total P&L']
        
                if pnl.loc['Total', 'Book Cost'] != 0:
                    pnl['Total P&L %'] = pnl['Total P&L'] / pnl.loc['Total', 'Book Cost'] * 100
                else:
                    pnl['Total P&L %'] = 0
        
                display(display_scrollable_df(pnl.sort_values(by='Weights', ascending=False).round(4)))                
                display(display_scrollable_df(trades))          
    
    def get_pnl_on_click(_):
        global book_cost,realized_pnl,profit_and_loss,trades
        
        url='https://github.com/niroojane/Risk-Management/raw/refs/heads/main/Trade%20History%20Reconstructed.xlsx'
        trade_history = read_excel_from_url(url)
        
        if trade_history is None:
            raise FileNotFoundError("Trade history could not be loaded. Execution stopped.")  
        loading_bar_pnl.value=0
        
        with holding_output:
            display(loading_bar_pnl)
            
        trades=Pnl_calculation.get_trade_in_usdt(trade_history)
        loading_bar_pnl.value+=100/3
        book_cost=Pnl_calculation.get_book_cost(trades)
        loading_bar_pnl.value+=100/3
        realized_pnl,profit_and_loss=Pnl_calculation.get_pnl(book_cost,trades)
        loading_bar_pnl.value+=100/3
        with holding_output:
            holding_output.clear_output()
            
        get_holdings(None)

    pnl_button=widgets.Button(description='Get P&L',button_style='info')
    pnl_button.on_click(get_pnl_on_click)
            
    position_button=widgets.Button(description='Get Positions',button_style='info')
    position_button.on_click(get_holdings)
    
    # --- layout ---

    allocation_ui=widgets.VBox([widgets.HBox([
            widgets.VBox([dropdown_asset, dropdown_sign, dropdown_limit]),
            widgets.VBox([add_constraint_btn, clear_constraints_btn, optimize_btn])]),
                                constraint_output,
        grid,
        widgets.HBox([button_add,button_clear,results_button])])

    constraint_ui = widgets.VBox([widgets.HBox([start_date_perf, end_date_perf,refresh_perf_button]),
                                               main_output,
        widgets.VBox([strat, rebalancing_frequency,benchmark_tracking_error,window_vol]),
        allocation_ui,strategy_output,
        widgets.HBox([start_date_perf, end_date_perf,refresh_perf_button]),
        output_returns,widgets.HBox([perf_output, drawdown_output]),widgets.HBox([vol_output,frontier_output])
    ])

    universe_ui = widgets.VBox([
        widgets.HBox([n_crypto, start_date, data_button]),
        scope_output,
        widgets.HBox([start_date_perf, end_date_perf,refresh_perf_button]),
        main_output,price_output
    ])
    
    calendar_perf=widgets.VBox([widgets.HBox([frequency_graph,fund,benchmark,graph_button]),calendar_output])
    positions_ui=widgets.VBox([widgets.HBox([position_button,pnl_button]),positions_output,holding_output])
    rebalancing_frequency_pnl=widgets.Dropdown(description='Frequency:', options=['Yearly','Quarterly','Monthly'], value='Quarterly')

    #------------Rik Tab------------#
    global var_scenarios, cvar_scenarios, fund_results
    risk_output = widgets.Output()
    
    start_date_perf_risk = widgets.DatePicker(value=start_perf_date, layout=widgets.Layout(width='350px'))
    end_date_perf_risk = widgets.DatePicker(value=datetime.date.today(), layout=widgets.Layout(width='350px'))
    # ---------- Ex-Ante / Risk Contribution ----------
    def update_fund_display(_):
        
        try:
            start_ts = pd.to_datetime(start_date_perf_risk.value)
            end_ts = pd.to_datetime(end_date_perf_risk.value)
        except Exception:
            with ex_ante_output:
                ex_ante_output.clear_output()   
                print("⚠️ Invalid start or end date.")
            with risk_output:
                risk_output.clear_output()   
            return
                
        if pd.isna(start_ts) or pd.isna(end_ts) or start_ts > end_ts: 
            with ex_ante_output:
                ex_ante_output.clear_output()   
                print("⚠️ Error with date range.")
            with risk_output:
                risk_output.clear_output()   
                return
    
        if dataframe.empty or returns_to_use.empty or grid.data.empty:
            with ex_ante_output:
                ex_ante_output.clear_output()   
                print("⚠️ Please compute optimization results first.")
            with risk_output:
                risk_output.clear_output()   
                return    
        range_prices = dataframe.loc[start_ts:end_ts]
    
        if range_prices.empty:
            with ex_ante_output:
                ex_ante_output.clear_output()   
                print("⚠️ No data available in selected date range.")
            with risk_output:
                risk_output.clear_output()   
            return
    
        range_returns = range_prices.pct_change().dropna()
 
        if grid.data.empty:
            with ex_ante_output:
                ex_ante_output.clear_output()
                print("⚠️ No Allocation.")
            with risk_output:
                risk_output.clear_output()   
            return     
            
        portfolio = RiskAnalysis(range_returns)
     
        selected_weights = grid.data.loc[selected_fund.value]
        
        decomposition = pd.DataFrame(portfolio.var_contrib_pct(selected_weights))*100
        quantities_rebalanced = rebalanced_portfolio(range_prices, selected_weights,frequency=rebalancing_frequency_pnl.value) / range_prices
        quantities_buy_hold = buy_and_hold(range_prices, selected_weights) / range_prices
        
        cost_rebalanced = rebalanced_book_cost(range_prices, quantities_rebalanced)
        cost_buy_and_hold = rebalanced_book_cost(range_prices, quantities_buy_hold)
        
        mtm_rebalanced = quantities_rebalanced * range_prices
        mtm_buy_and_hold = quantities_buy_hold * range_prices
        
        pnl_buy_and_hold=pd.DataFrame((mtm_buy_and_hold-cost_buy_and_hold).iloc[-1])
        pnl_buy_and_hold.columns=['Profit and Loss (Buy and Hold)']
        
        pnl_rebalanced=pd.DataFrame((mtm_rebalanced-cost_rebalanced).iloc[-1])
        pnl_rebalanced.columns=['Profit and Loss (Rebalanced)']
        
        profit_and_loss_simulated = pd.concat([pnl_buy_and_hold, pnl_rebalanced, decomposition], axis=1)
        profit_and_loss_simulated.loc['Total'] = profit_and_loss_simulated.sum(axis=0)
        profit_and_loss_simulated=profit_and_loss_simulated.fillna(0)
        
        with risk_output:
            risk_output.clear_output(wait=True)
            display(Markdown("### Performance and Risk Contribution"))
            display(display_scrollable_df(
                profit_and_loss_simulated.sort_values(by='Variance Contribution in %', ascending=False)
            ))

    def on_fund_change(change):
        if change['name'] == 'value' and change['new'] in grid.data.index:
            update_fund_display(change['new'])

    selected_fund.observe(on_fund_change)
    
    def on_freq_change(change):
        if change['name'] == 'value' and change['new'] in rebalancing_frequency_pnl.options:
            update_fund_display(change['new'])
    
    rebalancing_frequency_pnl.observe(on_freq_change,names='value')
         
    # ---------- Ex-Ante Metrics ----------
    ex_ante_output = widgets.Output()

    def ex_ante_metrics(bench_name):
        
        try:
            start_ts = pd.to_datetime(start_date_perf_risk.value)
            end_ts = pd.to_datetime(end_date_perf_risk.value)
        except Exception:
            with ex_ante_output:
                ex_ante_output.clear_output()   
                print("⚠️ Invalid start or end date.")

            with risk_output:
                risk_output.clear_output()   
            return

        if pd.isna(start_ts) or pd.isna(end_ts) or start_ts > end_ts: 
            with ex_ante_output:
                ex_ante_output.clear_output()   
                print("⚠️ Error with date range.")
            with risk_output:
                risk_output.clear_output()   
                return
    
        if dataframe.empty or returns_to_use.empty or grid.data.empty:
            with ex_ante_output:
                ex_ante_output.clear_output()   
                print("⚠️ Please compute optimization results first.")
            with risk_output:
                risk_output.clear_output()   
                return
    
        # ✅ Safe slicing
        range_prices = dataframe.loc[start_ts:end_ts]
    
        if range_prices.empty:
            with ex_ante_output:
                ex_ante_output.clear_output()   
                print("⚠️ No data available in selected date range.")
            return
    
        range_returns = range_prices.pct_change().dropna()
    
        if range_returns.empty:
            with ex_ante_output:
                ex_ante_output.clear_output()   
                print("⚠️ Not enough data to compute returns.")
            return
        
        vol_ex_ante = {}
        tracking_error_ex_ante = {}

        portfolio = RiskAnalysis(range_returns)
        
        for idx in grid.data.index:
            vol_ex_ante[idx] = portfolio.variance(grid.data.loc[idx])
            tracking_error_ex_ante[idx] = portfolio.variance(grid.data.loc[idx] - grid.data.loc[bench_name])

        data = {
            'Vol Ex Ante': vol_ex_ante,
            'Tracking Error Ex Ante': tracking_error_ex_ante
        }
        ex_ante_dataframe = pd.DataFrame(data)
        
        with ex_ante_output:
            ex_ante_output.clear_output(wait=True)
            display(Markdown("### Ex Ante Metrics"))
            display(display_scrollable_df(ex_ante_dataframe))

    def on_bench_change(change):
        if change['name'] == 'value' and change['new'] in grid.data.index:
            ex_ante_metrics(change['new'])
            
    def update_contrib_and_ex_ante(_):
        update_fund_display(None)
        ex_ante_metrics(selected_bench.value)
        
        
    ex_ante_metrics(selected_bench.value)
    selected_bench.observe(on_bench_change, names='value')

    end_date_perf_risk.observe(update_contrib_and_ex_ante)
    start_date_perf_risk.observe(update_contrib_and_ex_ante)

    # ---------- VaR / CVaR Simulation ----------
    var_output = widgets.Output()
    var_scenarios, cvar_scenarios, fund_results = {}, {}, {}

    stress_factor = widgets.BoundedFloatText(value=1.0, min=1.0, max=3.0, step=0.1, description='Stress Factor')
    iterations = widgets.BoundedIntText(value=10000, min=1000, max=100000, step=1, description='Iterations')
    num_scenarios = widgets.BoundedIntText(value=100, min=1, max=1000, step=1, description='Scenarios')
    var_centile = widgets.BoundedFloatText(value=0.05, min=0, max=1, step=0.01, description='VaR Centile')
    loading_bar_var = widgets.IntProgress(description='Loading scenarios...',min=0, max=100,style={'description_width': '150px'})

    def get_var_metrics(_):
        global var_scenarios, cvar_scenarios, fund_results
        
        try:
            start_ts = pd.to_datetime(start_date_perf_risk.value)
            end_ts = pd.to_datetime(end_date_perf_risk.value)
        except Exception:
            with var_output:
                var_output.clear_output()   
                print("⚠️ Invalid start or end date.")
            return
            
        with var_output:
            var_output.clear_output()   
            if pd.isna(start_ts) or pd.isna(end_ts) or start_ts > end_ts:
                print("⚠️ Error with date range.")
                return
            if returns_to_use.empty:
                print('⚠️Load Prices.')
                return
            if dataframe.empty or returns_to_use.empty or grid.data.empty:
                print("⚠️ Please compute optimization results first.")
                return

            horizon = 1 / 250
            spot = dataframe.iloc[-1]
            theta = 2
    
            distrib_functions = {
                'multivariate_distribution': (iterations.value, stress_factor.value),
                'gaussian_copula': (iterations.value, stress_factor.value),
                't_copula': (iterations.value, stress_factor.value),
                'gumbel_copula': (iterations.value, theta),
                'monte_carlo': (spot, horizon, iterations.value, stress_factor.value)
            }
    
            
            range_prices=dataframe.loc[start_ts:end_ts]
            range_returns=range_prices.pct_change()
            
            portfolio = RiskAnalysis(range_returns)
            
            var_scenarios, cvar_scenarios, fund_results = {}, {}, {}

            display(loading_bar_var)
            def process_index(index):
                vs, cvs = {}, {}
                for func_name, args in distrib_functions.items():
                    func = getattr(portfolio, func_name)
                    scenarios = {}
    
                    for i in range(num_scenarios.value):
                        if func_name == 'monte_carlo':
                            distrib = pd.DataFrame(func(*args)[1], columns=portfolio.returns.columns)
                        else:
                            distrib = pd.DataFrame(func(*args), columns=portfolio.returns.columns)
    
                        distrib = distrib * grid.data.loc[index]
                        distrib = distrib[distrib.columns[grid.data.loc[index] > 0]]
                        distrib['Portfolio'] = distrib.sum(axis=1)
    
                        results = distrib.sort_values(by='Portfolio').iloc[int(distrib.shape[0] * var_centile.value)]
                        scenarios[i] = results
    
                    scenario = pd.DataFrame(scenarios).T
                    mean_scenario = scenario.mean()
                    index_cvar = scenario['Portfolio'] < mean_scenario['Portfolio']
                    cvar = scenario.loc[index_cvar].mean()
    
                    vs[func_name] = mean_scenario
                    cvs[func_name] = cvar
    
                fund_result = {
                    'Value At Risk': mean_scenario.loc['Portfolio'],
                    'CVaR': cvar.loc['Portfolio']
                }
    
                return index, vs, cvs, fund_result
    
            # Threaded execution
            with ThreadPoolExecutor() as executor:
                futures = {executor.submit(process_index, idx): idx for idx in grid.data.index}
                for future in as_completed(futures):
                    idx, vs, cvs, fund_result = future.result()
                    var_scenarios[idx] = vs
                    cvar_scenarios[idx] = cvs
                    fund_results[idx] = fund_result
                    loading_bar_var.value += 100 / len(grid.data.index)
    
            loading_bar_var.value = 0
    
        display_var_results(selected_fund_var.value)
        loading_bar_var.value=0

    def display_var_results(fund_name):
        if fund_name not in var_scenarios:
            with var_output:
                var_output.clear_output(wait=True)
                print(f"⚠️ No VaR data found for '{fund_name}'. Run simulation first.")
            return

        columns = ['Multivariate', 'Gaussian Copula', 'T-Student Copula', 'Gumbel Copula', 'Monte Carlo']
        var_dataframe = pd.DataFrame(var_scenarios[fund_name])
        var_dataframe.columns = columns

        cvar_dataframe = pd.DataFrame(cvar_scenarios[fund_name])
        cvar_dataframe.columns = columns

        fund_results_dataframe = pd.DataFrame(fund_results).T

        with var_output:
            var_output.clear_output(wait=True)
            
            display(display_scrollable_df(fund_results_dataframe))

            display(Markdown(f"### VaR Results for **{fund_name}**"))

            display(display_scrollable_df(var_dataframe))

            display(Markdown(f"### CVaR Results for **{fund_name}**"))

            display(display_scrollable_df(cvar_dataframe))

    def update_var_metrics(change):
        if change['name'] == 'value' and change['new'] in var_scenarios:
            display_var_results(change['new'])

    selected_fund_var.observe(update_var_metrics, names='value')

    get_var_button = widgets.Button(description='Run Simulation', button_style='info')
    get_var_button.on_click(get_var_metrics)


    # ---------- Layout ----------
    ex_ante_ui = widgets.VBox([widgets.HBox([start_date_perf_risk, end_date_perf_risk]),
        widgets.VBox([selected_fund, selected_bench,rebalancing_frequency_pnl]),
        risk_output,
        ex_ante_output
    ])

    var_ui = widgets.VBox([widgets.HBox([start_date_perf_risk, end_date_perf_risk]),
        widgets.VBox([selected_fund_var, stress_factor, iterations, num_scenarios,var_centile, get_var_button]),
        var_output
    ])


    # ---------- Market Risk Metrics ----------

    pca_output=widgets.Output()
    pca_components=widgets.Output()
    start_date_market_risk = widgets.DatePicker(value=start_perf_date, layout=widgets.Layout(width='350px'))
    end_date_market_risk = widgets.DatePicker(value=datetime.date.today(), layout=widgets.Layout(width='350px'))
    
    def get_market_risk_metrics(_):
        
        try:
            start_ts = pd.to_datetime(start_date_market_risk.value)
            end_ts = pd.to_datetime(end_date_market_risk.value)
        except Exception:
            with pca_output:
                pca_output.clear_output()   
            with pca_components:
                pca_components.clear_output()   
                print("⚠️ Invalid start or end date.")
            return
        
        if pd.isna(start_ts) or pd.isna(end_ts) or start_ts > end_ts:
            with pca_output:
                pca_output.clear_output()   
            with pca_components:
                pca_components.clear_output()   
                print("⚠️ Error with date range.")
            return
        
        if returns_to_use.empty:

            with pca_output:
                pca_output.clear_output()            
            with pca_components:
                pca_components.clear_output()
                print('⚠️ Load Prices.')
            return
            
        market_tickers=[t for t in tickers if t in dataframe.columns]

        range_returns=returns_to_use.loc[start_ts:end_ts,market_tickers]
        portfolio=RiskAnalysis(range_returns)
    

        eigval,eigvec,portfolio_components=portfolio.pca(num_components=num_components.value)
        selected_components.options=portfolio_components.columns
        num_components.max=len(range_returns.columns)+1
        num_closest_to_pca.max=len(range_returns.columns)
        
        variance_explained=eigval/eigval.sum()
        variance_explained_dataframe=pd.DataFrame(variance_explained,index=portfolio_components.columns,columns=['Variance Explained'])
        
        pca_weight=dict((portfolio_components[selected_components.value]/(portfolio_components[selected_components.value]).sum()))
        pca_portfolio=pd.DataFrame(portfolio_components[selected_components.value]).sort_values(by=selected_components.value,ascending=False)
        
        historical_PCA=pd.DataFrame(np.array(list(pca_weight.values())).dot(np.transpose(portfolio.returns)),index=portfolio.returns.index,columns=['PCA'])
        historical_PCA=historical_PCA.dropna()
        historical_PCA.iloc[0]=0
        
        comparison=portfolio.returns.copy()
        comparison['PCA']=historical_PCA
        distances=np.sqrt(np.sum(comparison.apply(lambda y:(y-historical_PCA['PCA'])**2),axis=0)).sort_values()
        
        pca_similarity=comparison[distances.index[:num_closest_to_pca.value]]
        pca_similarity.iloc[0]=0
        pca_similarity=(1+pca_similarity).cumprod()*100

    
        with pca_components:
            
            pca_components.clear_output(wait=True)
            
            fig=px.bar(variance_explained_dataframe,title='Variance Explanation in %')
            fig.update_layout(plot_bgcolor="black", paper_bgcolor="black", font_color="white", width=800, height=400) 
            fig.update_traces(textfont=dict(family="Arial Narrow", size=15))

            fig2=px.bar(pca_portfolio,title='Eigen Weights')
            fig2.update_layout(plot_bgcolor="black", paper_bgcolor="black", font_color="white",width=800, height=400) 
            fig2.update_traces(textfont=dict(family="Arial Narrow", size=15))

            fig.show()
            fig2.show()
            
        with pca_output:
            pca_output.clear_output(wait=True)
            
            fig3=px.line((1+historical_PCA).cumprod()*100,title='Eigen Index', render_mode = 'svg')
            fig3.update_layout(plot_bgcolor="black", paper_bgcolor="black", font_color="white", width=800, height=400)
            fig3.update_traces(textfont=dict(family="Arial Narrow", size=15))

            fig4=px.line(pca_similarity,title='PCA Similarity', render_mode = 'svg')
            fig4.update_layout(plot_bgcolor="black", paper_bgcolor="black", font_color="white", width=800, height=400)
            fig4.update_traces(textfont=dict(family="Arial Narrow", size=15))

            fig3.show()
            fig4.show()


    start_date_market_risk.observe(get_market_risk_metrics)
    end_date_market_risk.observe(get_market_risk_metrics)

    asset_output_corr = widgets.Output()

    button_corr = widgets.Button(description="Show Correlation", button_style="success")
    
    window_corr=widgets.IntText(
    value=252,
    description='Rolling Correlation:',
    disabled=False,style={'description_width': '150px'}
    )

    
    def update_correlation(change=None):

        try:
            start_ts = pd.to_datetime(start_date_market_risk.value)
            end_ts = pd.to_datetime(end_date_market_risk.value)
        except Exception:
            with asset_output_corr:
                asset_output_corr.clear_output()
                print("⚠️ Invalid start or end date.")
            return
            
        if returns_to_use.empty:
            with asset_output_corr:
                asset_output_corr.clear_output()
                print('⚠️Load Prices.')
                return
            
        if dropdown_asset1.value==dropdown_asset2.value:
            with asset_output_corr:

                asset_output_corr.clear_output()
                print('⚠️Same asset')
                return    
            
        with asset_output_corr:
            asset_output_corr.clear_output()
            if pd.isna(start_ts) or pd.isna(end_ts) or start_ts > end_ts:
                print("⚠️ Error with date range.")
                return
   
        range_returns=returns_to_use.loc[start_ts:end_ts]
        pca_over_time=first_pca_over_time(returns=range_returns,window=window_corr.value)

        rolling_corr_output=widgets.Output()
        correlation_matrix=widgets.Output()
        pca_overtime_output=widgets.Output()
        
        with asset_output_corr:
            asset_output_corr.clear_output(wait=True)
            
            rolling_correlation = range_returns[dropdown_asset1.value].rolling(window_corr.value).corr(
                range_returns[dropdown_asset2.value]
            ).dropna()

            with rolling_corr_output:
                fig = px.line(rolling_correlation, title=f"{dropdown_asset1.value}/{dropdown_asset2.value} Correlation", render_mode = 'svg')
                fig.update_layout(plot_bgcolor="black", paper_bgcolor="black", font_color="white", width=800, height=400)
                fig.update_traces(textfont=dict(family="Arial Narrow", size=15))
    
                fig.show()

            with correlation_matrix:
                fig2 = px.imshow(range_returns.corr().round(2), title='Correlation Matrix',color_continuous_scale='blues', text_auto=True, aspect="auto")
                fig2.update_layout(plot_bgcolor="black", paper_bgcolor="black", font_color="white",width=800, height=400)
                fig2.update_traces(xgap=2, ygap=2)
                fig2.update_traces(textfont=dict(family="Arial Narrow", size=15))
                fig2.show()
            with pca_overtime_output:
                fig3=px.line(pca_over_time,title='First principal component (Variance Explained in %)', render_mode = 'svg')
                fig3.update_layout(plot_bgcolor="black", paper_bgcolor="black", font_color="white",width=800, height=400)
                fig3.update_layout(xaxis_title=None, yaxis_title=None)
                fig3.show()
                
            ui=widgets.HBox([rolling_corr_output,pca_overtime_output,correlation_matrix])
            display(ui)
            
    selected_components=widgets.Dropdown(options=['PC1'],description='Select PCA',style={'description_width': '150px'})
    num_components=widgets.BoundedIntText(min=1,max=5,value=5,description='PCA Components',style={'description_width': '150px'})
    num_closest_to_pca=widgets.BoundedIntText(min=1,max=20,value=5,description='PCA Closest',style={'description_width': '150px'})
    market_button=widgets.Button(description='Market Risk Analysis',button_style='info',style={'description_width': '150px'})
    market_button.on_click(get_market_risk_metrics)    
    correlation_button=widgets.Button(description='Get Correlation',button_style='info',style={'description_width': '150px'})
    correlation_button.on_click(update_correlation)

    market_ui=widgets.VBox([widgets.HBox([start_date_market_risk,
                                          end_date_market_risk,market_button]),
                            num_components,selected_components,num_closest_to_pca,
                            widgets.HBox([pca_components,pca_output])])
    
    correlation_ui=widgets.VBox([widgets.HBox([start_date_market_risk,end_date_market_risk,correlation_button]),dropdown_asset1,dropdown_asset2,window_corr,asset_output_corr])    
    
    global daily_pnl,pnl_history,historical_ptf,performance_ex_post,positions,quantities_holding
    
    daily_pnl=pd.DataFrame()
    pnl_history=pd.DataFrame()
    historical_ptf=pd.DataFrame()
    performance_ex_post=pd.DataFrame()
    positions=pd.DataFrame()
    quantities_holding=pd.DataFrame()
        
    def check_connection(_):
        global quantities_holding,positions
        url_positions='https://github.com/niroojane/Risk-Management/raw/refs/heads/main/Positions.xlsx'
        url_quantities='https://github.com/niroojane/Risk-Management/raw/refs/heads/main/Quantities.xlsx'
        
        with ex_post_perf:
            
            ex_post_perf.clear_output(wait=True)
            
            position = read_excel_from_url(url_positions,index_col=0)
            if position is None:
                raise FileNotFoundError("Positions.xlsx could not be loaded. Execution stopped.")
                print('Positions Not Found in Repository')
                
            quantities_history = read_excel_from_url(url_quantities,index_col=0)
            if quantities_history is None:
                raise FileNotFoundError("Quantities.xlsx could not be loaded. Execution stopped.")
                print('Quantities Not Found in Repository')
            
            # position=pd.read_excel('Positions.xlsx',index_col=0)
            positions,quantities_holding=Binance.get_positions_history(enddate=datetime.datetime.today())
            positions=positions.sort_index()
            positions.index=pd.to_datetime(positions.index)
            positions=pd.concat([position,positions])
            positions.index=pd.to_datetime(positions.index)
            positions=pd.concat([position,positions]).sort_index()
            positions=positions.loc[~positions.index.duplicated(keep='first'),:]
            positions['Total']=positions.loc[:,positions.columns!='Total'].sum(axis=1)
            
            # quantities_history=pd.read_excel('Quantities.xlsx',index_col=0)
            
            quantities_holding.index=pd.to_datetime(quantities_holding.index)
            quantities_holding=pd.concat([quantities_holding,quantities_history])
            quantities_holding=quantities_holding.loc[~quantities_holding.index.duplicated(),:]
        
            quantities_holding=quantities_holding.sort_index()

            start_date_perf_ex_post.value=positions.index[0].date()
            
    start_date_perf_ex_post = widgets.DatePicker(value=datetime.date.today(), layout=widgets.Layout(width='350px'))
    end_date_perf_ex_post = widgets.DatePicker(value=datetime.date.today(), layout=widgets.Layout(width='350px'))
    ex_post_perf=widgets.Output()
    ex_post_calendar=widgets.Output()
    
    fund_ex_post=widgets.Dropdown(value='Historical Portfolio',options=['Historical Portfolio','Fund'],description='Select Fund',style={'description_width': '150px'})
    benchmark_ex_post=widgets.Dropdown(value='Historical Portfolio',options=['Historical Portfolio','Fund'],description='Select Benchmark',style={'description_width': '150px'})
    frequency_graph_ex_post=widgets.Dropdown(options=['Year','Month'],value='Year',description='Select Frequency',style={'description_width': '150px'})
    calendar_button_ex_post=widgets.Button(description='Update', button_style='info')
    
    def show_graph_ex_post(_):
        
        try:
            start_ts = pd.to_datetime(start_date_perf_ex_post.value)
            end_ts = pd.to_datetime(end_date_perf_ex_post.value)
        except Exception:
            with ex_post_calendar:
                ex_post_calendar.clear_output()
                print("⚠️ Invalid start or end date.")
            return
    
        with ex_post_calendar:
            ex_post_calendar.clear_output()
    
            if pd.isna(start_ts) or pd.isna(end_ts) or start_ts > end_ts:
                print("⚠️ Error with date range.")
                return
    
            if pnl_history.empty:
                print("⚠️ P&L not computed.")
                return  
        cumulative_performance_ex_post=pd.DataFrame()
        
        if global_returns.empty:
            performance_ex_post=historical_ptf['Historical Portfolio'].copy()
            performance_ex_post=performance_ex_post.to_frame()
        else:
            performance_ex_post=historical_ptf['Historical Portfolio'].copy()
            performance_ex_post=pd.concat([performance_ex_post,global_returns],axis=1).sort_index()
        
            options = list(performance_ex_post.columns)
            fund_ex_post.options = options
            benchmark_ex_post.options = options
            
        with ex_post_calendar:
            return_and_vol_graph=widgets.Output()
            sharpe_and_te_graph=widgets.Output()
            
            ex_post_calendar.clear_output()
            
            if performance_ex_post.empty:
                print("⚠️ Load Ex Post Performance.")
                return
            if fund_ex_post.value==benchmark_ex_post.value:
                print("⚠️ Benchmark and Fund must be different.")
                return
            if performance_ex_post.empty or performance_ex_post.shape[1]<2:
                print("⚠️ No performance data available yet. Please run an optimization first.")
                return
                
            cumulative_performance_ex_post=performance_ex_post.loc[start_date_perf_ex_post.value:end_date_perf_ex_post.value].copy()
            cumulative_performance_ex_post.iloc[0]=0
            cumulative_performance_ex_post=(1+cumulative_performance_ex_post).cumprod()*100   
            
            graphs=get_calendar_graph(cumulative_performance_ex_post, 
                               freq=frequency_graph_ex_post.value, 
                               benchmark=benchmark_ex_post.value, 
                               fund=fund_ex_post.value)
            
            # for name, fig in graphs.items():
            #     fig.show()
            keys=list(graphs.keys())
            with return_and_vol_graph:
                graphs[keys[0]].show()
                graphs[keys[2]].show()
            with sharpe_and_te_graph:
                graphs[keys[1]].show()
                graphs[keys[3]].show()   

            ui=widgets.HBox([return_and_vol_graph,sharpe_and_te_graph])
            
            display(ui)
            
            update_ex_post_chart(None)

    calendar_button_ex_post.on_click(show_graph_ex_post)


    def update_ex_post_chart(_):

        try:
            start_ts = pd.to_datetime(start_date_perf_ex_post.value)
            end_ts = pd.to_datetime(end_date_perf_ex_post.value)
        except Exception:
            with ex_post_perf:
                ex_post_perf.clear_output()
                print("⚠️ Invalid start or end date.")

            return
    
        with ex_post_perf:
            ex_post_perf.clear_output()
    
            if pd.isna(start_ts) or pd.isna(end_ts) or start_ts > end_ts:
                print("⚠️ Error with date range.")
                return
    
            if pnl_history.empty:
                print("⚠️ P&L not computed.")
                return

        selected_cumulative_pnl=daily_pnl.loc[start_ts:end_ts,'Total'].copy()
        selected_cumulative_pnl.iloc[0]=0
        
        selected_history=pd.concat([selected_cumulative_pnl.cumsum(),pnl_history['Total'].loc[start_ts:end_ts]],axis=1)
        selected_history.columns=['Cumulative P&L','Total P&L']
        
        selected_daily_pnl=daily_pnl.loc[start_ts:end_ts].copy()
        selected_positions=positions.loc[start_ts:end_ts,"Total"]
        
        if global_returns.empty:
            performance_ex_post=historical_ptf['Historical Portfolio'].copy()
            performance_ex_post=performance_ex_post.to_frame()
        else:
            performance_ex_post=historical_ptf['Historical Portfolio'].copy()
            performance_ex_post=pd.concat([performance_ex_post,global_returns],axis=1).sort_index()  
        
        cumulative_performance_ex_post=performance_ex_post.loc[start_ts:end_ts].copy()
        cumulative_performance_ex_post.iloc[0]=0
        cumulative_performance_ex_post=(1+cumulative_performance_ex_post).cumprod()*100
        pnl_contribution=(pnl_history-pnl_history.shift(1)).loc[start_ts:end_ts]
        
        git_output=widgets.Output()
        
        def git_push(_):
            
            with git_output:
                git_output.clear_output(wait=True)
                
                quantities_holding.to_excel('Quantities.xlsx',index=False)
                positions.to_excel('Positions.xlsx')
                
                git.push_or_update_file(positions,'Positions')
                git.push_or_update_file(quantities_holding,'Quantities')
                    

        push_button=widgets.Button(description='Upload Files',button_style='success')
        push_button.on_click(git_push)

        expost_output=widgets.Output()
        expost_output1=widgets.Output()

        with ex_post_perf:
            ex_post_perf.clear_output(wait=True)

            with expost_output:
            
                fig=px.line(selected_positions,title='Portfolio Value', render_mode = 'svg')
                fig.update_layout(plot_bgcolor="black", paper_bgcolor="black", font_color="white",width=800, height=400)
                fig.update_layout(xaxis_title=None, yaxis_title=None)
                fig.show()
                
                fig2=px.line(selected_history,title='Cumulative P&L', render_mode = 'svg')
                fig2.update_layout(plot_bgcolor="black", paper_bgcolor="black", font_color="white",width=800, height=400)
                fig2.update_traces(visible="legendonly", selector=lambda t: not t.name in ['Cumulative P&L'])
    
                fig2.update_layout(xaxis_title=None, yaxis_title=None)
                fig2.show()            
            
            with expost_output1:

                fig3=px.line(cumulative_performance_ex_post,title='Cumulative Return', render_mode = 'svg')
                fig3.update_layout(plot_bgcolor="black", paper_bgcolor="black", font_color="white",width=800, height=400)
                fig3.update_traces(visible="legendonly", selector=lambda t: not t.name in ['Historical Portfolio'])
                fig3.update_layout(xaxis_title=None, yaxis_title=None)
                fig3.show()
                
                fig4 = px.bar(selected_daily_pnl, color=selected_daily_pnl['color'],
                     color_discrete_map={'green': 'green', 'red': 'red'},
                     title="Daily P&L")
                fig4.update_layout(plot_bgcolor="black", paper_bgcolor="black", font_color="white",width=800, height=400)
                fig4.update_layout(xaxis_title=None, yaxis_title=None,showlegend=False)
                fig4.show()
                
            ui=widgets.HBox([expost_output,expost_output1])
            
            display(ui)
            display(display_scrollable_df(pnl_contribution))
            display(push_button)
            display(git_output)
    
    def get_ex_post_returns(_):
        
        global daily_pnl,pnl_history,historical_ptf,performance_ex_post
 
        loading_bar.value=0

        with ex_post_perf:
            ex_post_perf.clear_output()
            
            display(loading_bar_pnl)
            display(loading_bar)
        if book_cost.empty:

            get_pnl_on_click(None)  
          
        quantities_tickers=list(quantities_holding.columns)
        daily_book_cost=book_cost.resample("D").last().dropna().sort_index()
        book_cost_history=pd.DataFrame()
        book_cost_history.index=set(daily_book_cost.index.append(quantities_holding.index))
        
        book_cost_history=book_cost_history.sort_index()
        cols= quantities_holding.columns[quantities_holding.columns!='USDCUSDT']
        
        for col in cols:
            
            book_cost_history[col]=daily_book_cost[col]
            
        book_cost_history=book_cost_history.ffill()
        book_cost_history=book_cost_history.loc[quantities_holding.index] 
        
        today = datetime.date.today()
        start_pnl=quantities_holding.index[0]
        days_total = (today - start_pnl.date()).days
        
        weights_ex_post=positions.copy()
        weights_ex_post=weights_ex_post.drop(columns=['USDTUSDT'])
        weights_ex_post=weights_ex_post.apply(lambda x: x/weights_ex_post['Total'])
        
        start_date=weights_ex_post.index[0].date()

        binance_data=get_price_threading(quantities_tickers,start_date)
        pnl_history=pd.DataFrame()
        pnl_history.index=quantities_holding.index
        pnl_history=pnl_history.sort_index()
        
        for col in cols:
            pnl_history[col]=quantities_holding[col]*(binance_data[col]-book_cost_history[col])
        pnl_history['Total']=pnl_history.sum(axis=1)
    
        
        daily_pnl=pnl_history['Total']-pnl_history['Total'].shift(1)
        daily_pnl=pd.DataFrame(daily_pnl)
        colors = ['green' if value >= 0 else 'red' for value in daily_pnl.values]
        
        daily_pnl['color'] = daily_pnl['Total'].apply(lambda v: 'green' if v >= 0 else 'red')
    
        binance_data_return=np.log(1+binance_data.pct_change(fill_method=None))
        weight_date=set(weights_ex_post.index)
        binance_date=set(binance_data_return.index)
        common_date=weight_date.intersection(binance_date)
        
        binance_data2=binance_data_return.loc[list(common_date)].copy().sort_index()
        weights_ex_post2=weights_ex_post.loc[list(common_date)].copy().sort_index()
        historical_ptf=pd.DataFrame()
        
        for col in binance_data:
            historical_ptf[col]=weights_ex_post2[col]*binance_data2[col]
            
        historical_ptf['Historical Portfolio']=historical_ptf.sum(axis=1)   
    
        if global_returns.empty:
            performance_ex_post=historical_ptf['Historical Portfolio'].copy()
            performance_ex_post=performance_ex_post.to_frame()
        else:
            performance_ex_post=historical_ptf['Historical Portfolio'].copy()
            performance_ex_post=pd.concat([performance_ex_post,global_returns],axis=1).sort_index()
        
            options = list(performance_ex_post.columns)
            fund_ex_post.options = options
            benchmark_ex_post.options = options
            fund_ex_post.value = 'Historical Portfolio'
            benchmark_ex_post.value ='Fund'

        update_ex_post_chart(None)
        show_graph_ex_post(None)
    
    ex_post_button=widgets.Button(description='Get P&L',button_style='info')
    ex_post_button.on_click(get_ex_post_returns)
    start_date_perf_ex_post.observe(update_ex_post_chart)
    end_date_perf_ex_post.observe(update_ex_post_chart)
    
    ex_post_ui=widgets.VBox([widgets.HBox([start_date_perf_ex_post,end_date_perf_ex_post,ex_post_button]),ex_post_perf])    
    calendar_ui_ex_post=widgets.VBox([widgets.HBox([frequency_graph_ex_post,fund_ex_post,benchmark_ex_post,calendar_button_ex_post]),ex_post_calendar])
    
    check_connection(None)

    investment_universe_tab = widgets.Output()
    strategy_tab = widgets.Output()
    positioning_tab = widgets.Output()
    ex_post_tab = widgets.Output()
    risk_analysis_tab = widgets.Output()
    market_risk_tab = widgets.Output()
    

    main_tabs = widgets.Tab(children=[
        investment_universe_tab,
        strategy_tab,
        ex_post_tab,
        risk_analysis_tab,
        market_risk_tab
    ])
    
    main_tabs.set_title(0, 'Investment Universe')
    main_tabs.set_title(1, 'Strategy')
    main_tabs.set_title(2, 'Current Portfolio')
    main_tabs.set_title(3, 'Risk Analysis')
    main_tabs.set_title(4, 'Market Risk')
    
    with investment_universe_tab:
        display(universe_ui)

    strategy_constraints_tab = widgets.Output()
    strategy_positions_tab = widgets.Output()
    strategy_returns_tab = widgets.Output()
    
    strategy_subtabs = widgets.Tab(children=[
        strategy_constraints_tab,
        strategy_positions_tab,
        strategy_returns_tab
    ])
    
    strategy_subtabs.set_title(0, 'Strategy')
    strategy_subtabs.set_title(1, 'Positioning')
    strategy_subtabs.set_title(2, 'Strategy Return')
    
    with strategy_tab:
        display(strategy_subtabs)
    
    with strategy_constraints_tab:
        display(constraint_ui)
    
    with strategy_positions_tab:
        display(positions_ui)
    
    with strategy_returns_tab:
        display(calendar_perf)


    pnl_sub_tab = widgets.Output()
    positions_subtab = widgets.Output()
    calendar_ex_post_subtab = widgets.Output()

    
    ex_post_subtabs = widgets.Tab(children=[
        pnl_sub_tab,
        positions_subtab,
        calendar_ex_post_subtab
    ])
    
    ex_post_subtabs.set_title(0, 'P&L')
    ex_post_subtabs.set_title(1, 'Positioning')
    ex_post_subtabs.set_title(2, 'Calendar Return')
    
    with ex_post_tab:
        display(ex_post_subtabs)
        
    with pnl_sub_tab:
        display(ex_post_ui)
        
    with positions_subtab:
        display(positions_ui)

    with calendar_ex_post_subtab:
        display(calendar_ui_ex_post)


    risk_contribution_tab = widgets.Output()
    var_tab = widgets.Output()
    
    risk_subtabs = widgets.Tab(children=[
        risk_contribution_tab,
        var_tab
    ])
    
    risk_subtabs.set_title(0, 'Risk Contribution')
    risk_subtabs.set_title(1, 'Value at Risk')
    
    with risk_analysis_tab:
        display(risk_subtabs)
    
    with risk_contribution_tab:
        display(ex_ante_ui)
    
    with var_tab:
        display(var_ui)
    
    market_risk_detail_tab = widgets.Output()
    correlation_tab = widgets.Output()
    
    market_risk_subtabs = widgets.Tab(children=[
        market_risk_detail_tab,
        correlation_tab
    ])
    
    market_risk_subtabs.set_title(0, 'Market Risk')
    market_risk_subtabs.set_title(1, 'Correlation')
    
    with market_risk_tab:
        display(market_risk_subtabs)
    
    with market_risk_detail_tab:
        display(market_ui)
    
    with correlation_tab:
        display(correlation_ui)

    display(main_tabs)

In [6]:
Binance=BinanceAPI(binance_api_key,binance_api_secret)
git=GitHub(token,repo_owner,repo_name,branch)
Pnl_calculation=PnL(binance_api_key,binance_api_secret)

In [7]:
display_crypto_app()

In [2]:
# from Crypto_App import display_crypto_app
# from Git import *
# from Binance_API import BinanceAPI
# from PnL_Computation import PnL

# Binance=BinanceAPI(binance_api_key,binance_api_secret)
# git=GitHub(token,repo_owner,repo_name,branch)
# Pnl_calculation=PnL(binance_api_key,binance_api_secret)
# display_crypto_app(Binance,Pnl_calculation,git)

In [9]:
# git.create_or_replace_notebook("Crypto_App.py")

In [6]:
git.create_or_replace_notebook("Crypto App.ipynb")

In [ ]:
# git.create_or_replace_notebook("Streamlit_App.py")